In [2]:
%load_ext autoreload
%autoreload 2

import os
import fnmatch
from pipeline.lda import LDABuilder
from pipeline.utils import *
import numpy as np
import pandas as pd

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [3]:
ldab = LDABuilder('all_the_news')
paths = ldab.paths

In [ ]:
dfs = []
for fp in [paths.data_file('articles' + str(i) + '.csv') for i in [3,1,2]]:
    print(fp)
    dfs.append( pd.read_csv(fp))
        
# dfs

In [ ]:
df = pd.concat(dfs).reset_index(drop=True)
df.drop(columns=['Unnamed: 0'], inplace=True)
# df['year'].astype(int)
# df['month'].astype(int)
df.info()

In [4]:
df2 = pd.read_csv(paths.corpus_meta_data)

In [16]:
len(df2[df2.partial_content.isna()])

86

In [17]:
len(df2[df2.url.isna()])

57011

In [ ]:
for i,parsed_doc in enumerate(read_doc_by_line(paths.trigram_corpus_filepath)):

    if i < 5 or (i >200000 and i < 200000+6) or i > len(df) - 7:
        print(parsed_doc[:50])
        print(df.loc[i].content[:50])

In [ ]:
import re
def partial_text(text):
    text = re.sub("[‹»›„‚“‟‘‛”’\"❛❜❝❞❮❯〝〞 ]{2,}", " ", text)
    return prep_whitespace(text)[:500].strip()

In [ ]:
print(df2.loc[90164].content)
# partial_text(df.loc[90164].content)

In [ ]:
df['partial_content'] = df['content'].map(lambda x: partial_text(x))
df.drop(columns=['content'], inplace=True)
df.to_csv(paths.corpus_meta_data)
df.info()

In [ ]:
df.head()

Note this is bad

In [ ]:
df[df.year.isin([np.nan, np.inf, -np.inf])]

In [ ]:
%%time
matrix = ldab.get_corpus_topics_matrix(recalculate=True)

In [ ]:
%%time
matrix = ldab.get_corpus_topics_matrix(75, recalculate=True)

In [ ]:
%%time
matrix = ldab.get_corpus_topics_matrix(100, recalculate=True)

In [ ]:
from pipeline.preprocessing import Preprocessor
prep = Preprocessor('all_the_news', preload_models=True)
meta_data = pd.read_csv(ldab.paths.corpus_meta_data)

In [ ]:
from random import *

index = randint(0,len(df)-1)
text = df.content[index]
lda = ldab.get_lda_model(100)
print(df.loc[index])
print(text[:1500])

In [ ]:
parsed_doc = prep.process_doc(' asdkjf')
corpus_dict = ldab.get_corpus_dict()
bow = corpus_dict.doc2bow(parsed_doc)
doc_topics = lda[bow]
topic_vec = ldab.create_topic_vec(lda.num_topics, doc_topics)
topic_vec

In [ ]:
n = 2
sorted_top = np.argpartition(topic_vec, -n)[-n:][::-1]

[print(lda.print_topic(i) + '\n') for i in sorted_top]
1

In [6]:
from sources import Sources
sources = Sources()
def append_bias(df):
    bias_map = sources.sources_bias_map
    bias = df['publication'].map(lambda x: bias_map.get(x, bias_map.get('The ' + x, bias_map.get(x + ' News'))))
    df['bias_label'] = bias.map(lambda b: sources.bias_display_names.get(b))
    df['bias_score'] = bias.map(lambda b: sources.bias_score.get(b))
    return df

In [15]:
# df2.head()
append_bias(df2[['publication']].sample(15))

,publication,bias_label,bias_score
62788,Breitbart,Far right,3
90158,Business Insider,Center-left,-1
36844,Washington Post,Center-left,-1
80545,CNN,Center-left,-1
26257,Reuters,Least Biased,0
108584,Talking Points Memo,Left,-2
79507,CNN,Center-left,-1
123781,New York Post,Right,2
117253,National Review,Right,2
43238,New York Times,Center-left,-1
